In [ ]:
!pip install pymongo
!pip install google-cloud-storage
!pip install fuzzywuzzy
!pip install "pymongo[srv]"

In [ ]:
from google.cloud import storage
import pandas as pd
import pandas as pd
import numpy as np
import requests
import json
from io import StringIO
from pymongo import MongoClient
from fuzzywuzzy import fuzz
import math
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from pymongo.errors import BulkWriteError

In [ ]:
# Specify the name of your bucket
bucket_name = 'restaurant-dataset'

# Authenticate using a service account key
# Replace 'path/to/your/service-account-key.json' with the path to your service account key file
storage_client = storage.Client.from_service_account_json('nyc-restaurants-419717-6cf4f9e6c135.json')

# Access the specified bucket
bucket = storage_client.bucket(bucket_name)

In [ ]:
# Specify the folder name within the bucket
folder_name = 'df_raw'

#specify name of the CSV file in the bucket
blob = bucket.blob (f'{folder_name}/StatenIslandBronxInspect.csv')

# Download the CSV file
blob.download_to_filename('StatenIslandBronxInspect.csv')

# Read the CSV file into a DataFrame
statenislandbronx_df = pd.read_csv('StatenIslandBronxInspect.csv')

# Display the DataFrame
statenislandbronx_df.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
0,50110282,KARIBBEAN KRAVE,Bronx,65,WESTCHESTER SQ,10461.0,7186736759,Caribbean,08/12/2022,No violations were recorded at the time of thi...,...,Administrative Miscellaneous / Initial Inspection,40.841234,-73.842973,210.0,18.0,20000.0,2097357.0,2.040720e+09,BX37,NaN
1,41450634,YUNG HSIN RESTAURANT,Bronx,147577,BEACH AVENUE,NaN,7184095984,Chinese,05/17/2023,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,0.000000,0.000000,NaN,NaN,NaN,NaN,2.000000e+00,NaN,NaN
2,41459414,NEW YUNG HONG,Bronx,271,WEST KINGSBRIDGE ROAD,10463.0,7185490030,Chinese,06/24/2019,No violations were recorded at the time of thi...,...,Cycle Inspection / Initial Inspection,40.872850,-73.905127,208.0,14.0,27300.0,2015865.0,2.032590e+09,BX28,NaN
3,50150619,GOTHAM STADIUM TENNIS CENTER CAFE,Bronx,725,EXTERIOR STREET,10451.0,9149800750,NaN,01/01/1900,NaN,...,NaN,40.823458,-73.931021,204.0,8.0,6300.0,2097176.0,2.025390e+09,BX63,NaN
4,41232292,PIZZA CHEF,Bronx,564,WEST 235 STREET,10463.0,7187964500,Pizza,04/18/2022,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.885708,-73.910276,208.0,11.0,29700.0,2084102.0,2.057870e+09,BX29,NaN


In [ ]:
statenislandbronx_df_cleaned = statenislandbronx_df.dropna(subset=['DBA'])
statenislandbronx_df_unique = statenislandbronx_df_cleaned.drop_duplicates(subset=['DBA'], keep='first')
StatenIslandBronx_df_sorted = statenislandbronx_df_unique.sort_values(by=['Latitude', 'Longitude'], ascending=False)
StatenIslandBronx_df_sorted.head()
print(StatenIslandBronx_df_sorted.shape)

(2796, 27)


In [ ]:
# Initialize an empty DataFrame to accumulate Yelp data
df_yelp = pd.DataFrame()

#specify name of the CSV file in the bucket
blob = bucket.blob (f'{folder_name}/yelp_data_clean.csv')

# Download the CSV file
blob.download_to_filename('yelp_data_clean.csv')

# Read the CSV file into a DataFrame
df_yelp_restaurants = pd.read_csv('yelp_data_clean.csv')

# Display the DataFrame
print(df_yelp_restaurants.shape)

(8266, 17)


In [ ]:
# Connect to MongoDB (update the connection string as necessary)
uri = "XXXX"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'),connect=False)
db = client['yelpdata']
collection = db['yelpratings']

In [ ]:
# Create an index on the 'id' to prevent duplicate record
# index_id = collection.create_index([('id', 1)], unique=True)
# print(f"Index created: {index_id}")

# Create an index on the 'name' field
# index_name = collection.create_index([('name', 1)])  # '1' for ascending order
# print(f"Index created: {index_name}")

In [ ]:
!curl ipecho.net/plain

35.193.71.180

In [ ]:
#convert dataframe to json array and send it to the mongodb only run this once.
#json = df_yelp_restaurants.to_dict(orient='records')
#result = collection.insert_many(json)

In [ ]:
# Mock function simulating the API call, returning multiple restaurants
def query_api(Latitude, Longitude):
  TOKEN = "YYYYYYY"
  headers = { "accept": "application/json", "Authorization": f"Bearer {TOKEN}"}
  url = f"https://api.yelp.com/v3/businesses/search?location=NYC&latitude={Latitude}&longitude={Longitude}&term=restaurants&radius=100&sort_by=review_count&limit=50&offset=1"
  response = requests.get(url, headers=headers)
  if response.status_code == 200:
    data = response.json()
    business_names = [business['name'] for business in data['businesses']]
  return data['businesses'], business_names

In [ ]:
index_to_skip = []
threshold = 50

restaurants_unique_api = list(collection.find({}, {'name': 1, 'rating': 1, 'review_count': 1}))
len(restaurants_unique_api)

14270

In [ ]:
StatenIslandBronx_df_sorted.shape

(2797, 29)

In [ ]:
# Use iterrows() to iterate through DataFrame rows
for index, row in StatenIslandBronx_df_sorted.iterrows():
    if index in index_to_skip and len(index_to_skip) > 0 :
      continue
    # Check if the current restaurantName is in the set of restaurants to remove
    restaurant_name = row['DBA']
    print(restaurant_name)
    if len(restaurants_unique_api) > 0:
      for restaurant in restaurants_unique_api:
        # Check against each name in the DataFrame
        match_score = fuzz.token_set_ratio(restaurant_name, restaurant['name'])
        if match_score >= threshold:
          (f"{restaurant_name} is in mongo")
          StatenIslandBronx_df_sorted.loc[index, 'yelp_rating'] = restaurant['rating']
          StatenIslandBronx_df_sorted.loc[index, 'yelp_review_count'] = restaurant['review_count']
          break

    # Call the API for each row's latitude and longitude not in restaurants_unique_api
    api_response_restaurants, restaurant_list = query_api(row['Latitude'], row['Longitude'])
    print(api_response_restaurants)

    if(len(api_response_restaurants) > 0):
      # Send to api_response_restaurants to mongodb or anything because I want to keep those values for future references
      # Also making sure that I am skipping any duplicate
      try:
        result = collection.insert_many(api_response_restaurants, ordered=False)
        print("Inserted IDs:", result.inserted_ids)
      except BulkWriteError as e:
        print("Bulk write error occurred:", e.details)

      for restaurant in api_response_restaurants:
        match_score = fuzz.token_set_ratio(restaurant_name, restaurant['name'])
        if match_score >= threshold:
          # Append the matching restaurant's rating and review count
          print(f"Rating for '{restaurant_name}': {restaurant['rating']}")
          print(f"Review Count for '{restaurant_name}': {restaurant['rating']}")
          StatenIslandBronx_df_sorted.loc[index, 'yelp_rating'] = restaurant['rating']
          StatenIslandBronx_df_sorted.loc[index, 'yelp_review_count'] = restaurant['review_count']
          break
    index_to_skip.append(index)
    restaurants_unique_api = list(collection.find({}, {'name': 1, 'rating': 1, 'review_count': 1}))

COLLEGE OF MOUNT ST.VINCENT (HUDSON HEIGHTS; 1ST FL)
[]
COLLEGE OF MOUNT SAINT VINCENT
[]
COLLEGE OF MOUNT SAINT VINCENT (Mag's Kitchen)
[]
KING'S WOK
[{'id': 'TuljfEsXH7kNAllekFTuBw', 'alias': 'madaba-deli-ii-bronx', 'name': 'Madaba Deli II', 'image_url': '', 'is_closed': False, 'url': 'https://www.yelp.com/biz/madaba-deli-ii-bronx?adjust_creative=26xVZ_BFf1IKrXeYmiYOWw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=26xVZ_BFf1IKrXeYmiYOWw', 'review_count': 3, 'categories': [{'alias': 'delis', 'title': 'Delis'}], 'rating': 1.7, 'coordinates': {'latitude': 40.910045, 'longitude': -73.897186}, 'transactions': ['pickup'], 'price': '$', 'location': {'address1': '6655 Broadway', 'address2': '', 'address3': '', 'city': 'Bronx', 'zip_code': '10471', 'country': 'US', 'state': 'NY', 'display_address': ['6655 Broadway', 'Bronx, NY 10471']}, 'phone': '+17185436092', 'display_phone': '(718) 543-6092', 'distance': 78.8286498848691}, {'id': '7Bnv8IIFdlCXS1k96QfE-A', 'alias': '

UnboundLocalError: local variable 'data' referenced before assignment

In [ ]:
print(index_to_skip)

[1752, 5462, 6371, 646, 11993, 545, 2590, 7194, 2418, 8122, 1383, 5368, 127, 2629, 11034, 2098, 2424, 4097, 10889, 2126, 8225, 2783, 4343, 62, 971, 3326, 2316, 8815, 4729, 389, 1343, 1686, 10607, 1562, 658, 19165, 3015, 2674, 332, 3491, 1491, 908, 19217, 4775, 17038, 1102, 2034, 531, 2506, 1715, 711, 14868, 18049, 2899, 323, 865, 1925, 4344, 271, 2125, 462, 1920, 1212, 3156, 543, 3658, 3687, 104, 714, 3272, 14193, 2503, 527, 7548, 607, 13864, 909, 7361, 6158, 1069, 12834, 745, 895, 1493, 1860, 4770, 4736, 282, 536, 2132, 738, 5186, 1888, 1067, 1419, 188, 2402, 65, 33, 468, 4459, 951, 1522, 99, 570, 1430, 860, 2344, 310, 2155, 380, 540, 7444, 1437, 5221, 14099, 8629, 1050, 42, 1265, 228, 914, 8091, 12, 1979, 2626, 9534, 9113, 546, 17, 6459, 2828, 1331, 999, 6736, 2455, 1778, 4849, 297, 1973, 419, 491, 18339, 319, 1704, 2406, 277, 1480, 1565, 642, 15753, 2265, 2858, 774, 2967, 6597, 3101, 1532, 5491, 1468, 8171, 1336, 1075, 14691, 507, 5321, 14453, 1408, 247, 1207, 6164, 576, 333, 552, 1

In [ ]:
StatenIslandBronx_df_sorted[['DBA', 'yelp_rating', 'yelp_review_count']]

,DBA,yelp_rating,yelp_review_count
1752,COLLEGE OF MOUNT ST.VINCENT (HUDSON HEIGHTS; 1...,3.2,194.0
5462,COLLEGE OF MOUNT SAINT VINCENT,4.0,20.0
6371,COLLEGE OF MOUNT SAINT VINCENT (Mag's Kitchen),0.0,0.0
646,KING'S WOK,3.7,529.0
11993,Marianne Pizza Cafe,2.9,80.0
...,...,...,...
21619,THE LOCAL,NaN,NaN
21851,HOOK & REEL,NaN,NaN
22402,DA NOI II,NaN,NaN
23834,WAR MEMORIAL ICE SKATING RINK,NaN,NaN


In [ ]:
StatenIslandBronx_df_sorted.to_csv("statenislandbronxyelp.csv")